# Experiment 01: Band configurations

In this experiment we will study how different band configurations affect the training process and end results.

### Experiment variations:

- E01-8: All 8 MS WorldView-2 bands is used.
- E01-6: The 6 MS WorldView-2 bands that overlap the PAN band (RGB+Y+RedEdge+NIR1)
- E01-4: The 4 MS WorldView-2 bands that are also available in GeoEye-1 (MS1 array RGB+NIR1)
 - In this variation we will also validate performance on the GeoEye-1 validation set
- E01-3: Only the 3 RGB bands from WorldView-2
 - Also validated on the GeoEye-1 validation set
 
### The notebook is divided into the following main sections:
1. Imports and configuration parameters
2. Tile generation (sampling of tiles from the satellite images)
3. Tile input pipelines (`tf.dataset` objects reading tiles from disk)
4. Building of models
5. Pretraining with L1 loss
6. Build the full ESRGAN model
7. GAN-training with L1 + Percep + GAN loss
8. Inspection of results

Training history is logged with TensorBoard.

## 1. Imports and configuration parameters

In [1]:
from modules.helpers import *
from modules.tile_generator import *
from modules.matlab_metrics import *
from modules.image_utils import *
from modules.tile_input_pipeline import *
from modules.models import *

from modules.logging import *
from modules.train import *

# Check GPUs and enable dynamic GPU memory use:",
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            # Prevent TensorFlow from allocating all memory of all GPUs:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [2]:
### MAIN SETTINGS ###############################################################################################
EXPERIMENT_NAMES = ['e01-8', 'e01-6', 'e01-4', 'e01-3']

# Select experiment variation to be run in THIS notebook:
EXPERIMENT = EXPERIMENT_NAMES[2]

# Turn on and off certain time consuming processes in the notebook:
GENERATE_TILES = False   # This should only be done once in experiment 01. All variations will read from the same
TILE_DENSITY_MAPS = False  # Loops through all tiles and compute density maps of where tiles have been sampled
CALCULATE_STATS = False  # Loops through all tiles and calculate mean and sd. Used for scaling
PRETRAIN = False          # Step 1 of the training process
GAN_TRAIN = True        # Step 2 of the training process

# Load metadata dataframe "meta" from repository root. 
# This dataframe keeops track of images and is used and updated throughout the notebook
meta = load_meta_pickle_csv('.', 'metadata_df', from_pickle=True)
#################################################################################################################

### PATHS #######################################################################################################
DATA_PATH = 'data/toulon-laspezia'
DATA_PATH_TILES = 'data/toulon-laspezia-tiles/e01'
DATA_PATH_TILES_P = {'train': DATA_PATH_TILES + '/train', 
                     'val': DATA_PATH_TILES + '/val', 
                     'test': DATA_PATH_TILES + '/test'}
LOGS_DIR = 'logs/' # Path to tensorboard logs and model checkpoint saves
LOGS_EXP_DIR = LOGS_DIR + EXPERIMENT
#################################################################################################################

### TILE GENERATION #############################################################################################
SENSORS_GENERATE = ['WV02', 'GE01']
AREAS_GENERATE = ['La_Spezia', 'Toulon']
meta = subset_by_areas_sensor(meta, areas=AREAS_GENERATE, sensors=SENSORS_GENERATE)
print('Sensors to generate tiles from:', SENSORS_GENERATE)
print('Areas to generate tiles from:', AREAS_GENERATE)

# Count images in partitions (train/val/test):
N_IMAGES_TOTAL = count_images(meta)
N_IMAGES = {'train': count_images_in_partition(meta, 'train'), 
            'val': count_images_in_partition(meta, 'val'), 
            'test': count_images_in_partition(meta, 'test')}
assert N_IMAGES_TOTAL == sum(N_IMAGES.values())  # Verify that different ways of counting adds up
print('Number of images in partitions', N_IMAGES)
print('Total number of images:', N_IMAGES_TOTAL)

TILES_PER_M2 = {'train': 2.0, 
                'val': 2.0, 
                'test': 2.0}

# Settings for whether to send every tile generated through a sea and cloud classifier
# This is useful if images consist of a lot of sea and clouds and you want to reduce the number of tiles
# with such monotone and less meaningful content. Classifier is trained on 2500 labeled tiles of various sizes
# where only tiles COMPLETELY covered by sea and/or clouds have been labelled "cloud/sea". 
# Validation accuracy around 0.95
CLOUD_SEA_REMOVAL = True
CLOUD_SEA_WEIGHTS_PATH = 'models/cloud-sea-classifier/cloudsea-effb0-augm-bicubic-pan-0.0005--200-0.129130.h5'
# Cutoff at inference time. Tiles with (quasi)-prob higher than cutoff will be classified as cloud and or sea:
CLOUD_SEA_PRED_CUTOFF = 0.95
# Setting to keep a certain proportion of cloud/sea tiles through the filter:
CLOUD_SEA_KEEP_RATE = 0.10

# GE01 images has some slight variations in resolution 0.5 +-0.05 m per pixel while WV02 is fixed at 0.5m
# Setting this to True will resize to as close as possible to 0.5m
# Not used in this notebook, but function is ready for use in module tile_generator.py
RESIZE_TO_PIXEL_SIZE = False
if RESIZE_TO_PIXEL_SIZE:
    RESIZE_RESAMPLING_METHOD = 'nearest'  # 'nearest', 'bicubic', 'bilinear'
#################################################################################################################

### SENSORS AND AREA EXPERIMENT SELECTION #######################################################################
# Sensors used in which experiment variation
SENSORS_EXP = {'e01-8': {'train': 'WV02', 'val': ['WV02'], 'test': ['WV02']}, 
               'e01-6': {'train': 'WV02', 'val': ['WV02'], 'test': ['WV02']}, 
               'e01-4': {'train': 'WV02', 'val': ['WV02', 'GE01'], 'test': ['WV02', 'GE01']}, 
               'e01-3': {'train': 'WV02', 'val': ['WV02', 'GE01'], 'test': ['WV02', 'GE01']}}
SENSORS = SENSORS_EXP[EXPERIMENT]

# Areas used in which experiment variation
AREAS_EXP = {'e01-8': {'train': AREAS_GENERATE, 'val': AREAS_GENERATE, 'test': AREAS_GENERATE}, 
             'e01-6': {'train': AREAS_GENERATE, 'val': AREAS_GENERATE, 'test': AREAS_GENERATE}, 
             'e01-4': {'train': AREAS_GENERATE, 'val': AREAS_GENERATE, 'test': AREAS_GENERATE}, 
             'e01-3': {'train': AREAS_GENERATE, 'val': AREAS_GENERATE, 'test': AREAS_GENERATE}}
AREAS = AREAS_EXP[EXPERIMENT]
#################################################################################################################

### TILE DIMENSIONS #############################################################################################
# Note larger size of val and test. This is needed for sensible calculation of Ma, NIQE and PI calculation
SR_FACTOR = 4
MS_SIZE = {'train': 32, 'val': 128, 'test': 128}
PAN_SIZE = {'train': MS_SIZE['train'] * SR_FACTOR, 
            'val': MS_SIZE['val'] * SR_FACTOR, 
            'test': MS_SIZE['test'] * SR_FACTOR}
print('MS (LR) tile size:', MS_SIZE)
print('PAN (HR) tile size:', PAN_SIZE)
print('SR factor:', SR_FACTOR)
#################################################################################################################

### BAND (CHANNEL) CONFIGURATIONS ###############################################################################
# This is the essence of experiment 01
# Selection of bands is done in the tile input pipeline

# Selecting bands from the 8 bands of WV02:
WV02_FULL_BAND_CONFIG = get_sensor_bands('WV02', meta)
WV02_EXP_BAND_CONFIGS = {'e01-8': WV02_FULL_BAND_CONFIG,                          # 8 (all) bands
                        'e01-6': {k:v for (k,v) in WV02_FULL_BAND_CONFIG.items()  # 6 bands (BGYR+RE+NIR)
                                  if k not in ['Coastal', 'NIR2']}, 
                        'e01-4': {k:v for (k,v) in WV02_FULL_BAND_CONFIG.items()  # 4 bands (BGR+NIR)
                                  if k in ['Blue', 'Green', 'Red', 'NIR']},
                        'e01-3': {k:v for (k,v) in WV02_FULL_BAND_CONFIG.items()  # 3 bands (BGR)
                                  if k in ['Blue', 'Green', 'Red']}}
MS_BANDS_WV02_CONFIG = WV02_EXP_BAND_CONFIGS[EXPERIMENT]
if EXPERIMENT == 'e01-8':
    # We set this to 'all' in order to not pass e01-8 tiles through a band selection function (no reason to)
    MS_BANDS_WV02_IDXS = 'all' 
else:
    # For the other experiment variations we need lists of indices of the bands to be selected
    MS_BANDS_WV02_IDXS = list(MS_BANDS_WV02_CONFIG.values())

N_MS_BANDS = len(MS_BANDS_WV02_CONFIG.values()) # The number of MS bands in this experiment variation

# Selecting bands from the 4 bands of GE01:
GE01_FULL_BAND_CONFIG = get_sensor_bands('GE01', meta)                            
GE01_EXP_BAND_CONFIGS = {'e01-8': {None: None},                                   # not enough bands in GE01
                        'e01-6': {None: None},                                    # not enough bands in GE01
                        'e01-4': GE01_FULL_BAND_CONFIG,                           # 4 (all) bands (BGR+NIR)
                        'e01-3': {k:v for (k,v) in GE01_FULL_BAND_CONFIG.items()  # 3 bands (BGR)
                                  if k not in ['NIR']}}
MS_BANDS_GE01_CONFIG = GE01_EXP_BAND_CONFIGS[EXPERIMENT]
if EXPERIMENT == 'e01-4':
    MS_BANDS_GE01_IDXS = 'all'
else:
    MS_BANDS_GE01_IDXS = list(MS_BANDS_GE01_CONFIG.values())
print('MS (LR) Band Config WV02:', MS_BANDS_WV02_CONFIG)
print('MS (LR) Band Config GE01:', MS_BANDS_GE01_CONFIG)

N_PAN_BANDS = 1 # Obviously only 1 panchromatic band
#################################################################################################################

### MODEL PARAMETERS ############################################################################################
BATCH_SIZE = {'train': 16, 'val': 8, 'test': 8}
print('Batch sizes:', BATCH_SIZE)

# RRDB Generator Model parameters 
N_BLOCKS = 16 # 23 - Deeper means potential to capture more complex relationships, at the cost of training time
N_FILTERS = 64 # Baseline setting that is not tinkered with in this repository
#################################################################################################################

### PRETRAINING SETTINGS ########################################################################################
PRE_EPOCHS = 400
PRE_TRAIN_STEPS = 1000  # per epoch
PRE_VAL_STEPS = 250     # per epoch
print('Pretraining - Total steps:', PRE_EPOCHS * PRE_TRAIN_STEPS)

# Number of batches to save every epoch in TensorBoard
TRAIN_N_BATCHES_SAVE = 2
VAL_N_BATCHES_SAVE = 2

# Optimizer settings:
PRETRAIN_LOSS = 'l1'    # Official
PRETRAIN_LR = 5e-5      # Tuned and found stable for this particular experiment
#PRETRAIN_LR = 0.0002   # Official
PRETRAIN_BETA_1 = 0.9   # Official
PRETRAIN_BETA_2 = 0.999 # Official
# Note: Official implementation also uses stepwise learning rate scheduler. 
# This is avoided here as it is deemed not central to the experiment to "squeeze" out last performance and it 
# complicates comparisons between experiment variations
#################################################################################################################

### GAN TRAINING SETTINGS #######################################################################################
GAN_EPOCHS = 400
GAN_TRAIN_STEPS = 1000
GAN_VAL_STEPS = 250
# Proportion of val batches that will go through ma and niqe metric calculation
MA_NIQE_PROPORTION = 0.04  # The calculation is very time consuming
print('GAN training - Total steps:', GAN_EPOCHS * GAN_TRAIN_STEPS)

# Weights for each loss in the composite loss function
G_LOSS_PIXEL_W = 0.01       # Official
G_LOSS_PERCEP_W = 1.0       # Official
G_LOSS_GENERATOR_W = 0.005  # Official

# Optimizer settings:
#GAN_G_LR = 1e-4 # Official
#GAN_D_LR = 1e-4 # Official
GAN_G_LR = 2e-5
GAN_D_LR = 2e-5
G_BETA_1, D_BETA_1 = 0.9, 0.9      # Official
G_BETA_2, D_BETA_2 = 0.999, 0.999  # Official
# Note: Official implementation also uses stepwise learning rate scheduler. 
# This is avoided here as it is deemed not central to the experiment to "squeeze" out last performance and it 
# complicates comparisons between experiment variations

# Path to the pretraining weights that is the starting point of GAN training:
PRETRAIN_WEIGHTS_PATHS = {'e01-8': LOGS_EXP_DIR + '/models/' + 'e01-8-pre_20210116-194500/' + 'e01-8-pre-400.h5', 
                          'e01-6': LOGS_EXP_DIR + '/models/' + 'e01-6-pre_20210122-091421/' + 'e01-6-pre-400.h5', 
                          'e01-4': LOGS_EXP_DIR + '/models/' + 'e01-4-pre_20210119-101939/' + 'e01-4-pre-400.h5', 
                          'e01-3': LOGS_EXP_DIR + '/models/' + 'e01-3-pre_20210124-153415/' + 'e01-3-pre-400.h5'}
PRETRAIN_WEIGHTS_PATH = PRETRAIN_WEIGHTS_PATHS[EXPERIMENT]
                         
#################################################################################################################

### MATLAB METRICS ##############################################################################################
# Calculate Ma, NIQE and Perceptual Index (PI) metrics on the validation set(s) during GAN training:
# PI was metric used in PIRM2018 competition https://github.com/roimehrez/PIRM2018
METRIC_MA = False
METRIC_NIQE = False
if METRIC_MA and METRIC_NIQE:
    METRIC_PI = True
else:
    METRIC_PI = False

# The number of pixels to be shaved off the border of the tile before calculating Ma/NIQE/PI (ignore border effects)
SHAVE_WIDTH = 4 # Official (as used in PIRM2018 evaluation)
# Ma/NIQE/PI calculation is done with official matlab repositories through MATLAB Engine API for Python
MATLAB_PATH = 'modules/matlab' # path to repositories
print('MATLAB Metrics:')
print('Ma:', METRIC_MA)
print('NIQE:', METRIC_NIQE)
print('Perceptual Index (PI):', METRIC_PI)

#################################################################################################################

Sensors to generate tiles from: ['WV02', 'GE01']
Areas to generate tiles from: ['La_Spezia', 'Toulon']
Number of images in partitions {'train': 22, 'val': 19, 'test': 21}
Total number of images: 62
MS (LR) tile size: {'train': 32, 'val': 128, 'test': 128}
PAN (HR) tile size: {'train': 128, 'val': 512, 'test': 512}
SR factor: 4
MS (LR) Band Config WV02: {'Blue': 1, 'Green': 2, 'Red': 4, 'NIR': 6}
MS (LR) Band Config GE01: {'Blue': 0, 'Green': 1, 'Red': 2, 'NIR': 3}
Batch sizes: {'train': 16, 'val': 8, 'test': 8}
Pretraining - Total steps: 400000
GAN training - Total steps: 400000
MATLAB Metrics:
Ma: False
NIQE: False
Perceptual Index (PI): False


## 2. Tile generation

### 2.1 Image resizing

Function `resize_sat_img_to_new_pixel_size` available in `modules.tile_generator`. Not used in this notebook

### 2.2 Tile allocation

We allocate `n_tiles` to each satellite image in proportion to the area covered by the satellite image. We adjust `n_tiles` by the argument `tiles_per_m2`. If `tiles_per_m2=1.0` then `n_tiles` is set deterministically to a value so that a square meter of satellite image is expected to be covered by `1.0` tile.

In [3]:
if GENERATE_TILES:
    meta = allocate_tiles_by_expected(meta, 
                                      override_pan_pixel_size=RESIZE_TO_PIXEL_SIZE,
                                      by_partition=True, 
                                      tiles_per_m2_train_val_test=(TILES_PER_M2['train'], 
                                                                   TILES_PER_M2['val'], 
                                                                   TILES_PER_M2['test']),
                                      pan_tile_size_train_val_test=(PAN_SIZE['train'], 
                                                                    PAN_SIZE['val'], 
                                                                    PAN_SIZE['test']),
                                      new_column_name='n_tiles')
else:
    # Load meta dataframe that was updated at tile generation time
    meta = load_meta_pickle_csv(DATA_PATH_TILES, 'metadata_tile_allocation', from_pickle=True)

n_tiles = {'train': count_tiles_in_partition(meta, 'train'),
           'val': count_tiles_in_partition(meta, 'val'), 
           'test':  count_tiles_in_partition(meta, 'test')}
n_tiles_total = count_tiles(meta)
assert n_tiles_total == sum(n_tiles.values())
print('Number of tiles per partition:')
print(n_tiles)
print('Total number of tiles:', n_tiles_total)

Number of tiles per partition:
{'train': 129221, 'val': 8113, 'test': 9293}
Total number of tiles: 146627


### 2.3 Tile generation to disk

In [4]:
if GENERATE_TILES:
    meta = generate_all_tiles(meta, 
                              save_dir=DATA_PATH_TILES, 
                              sr_factor=SR_FACTOR, 
                              by_partition=True,
                              ms_tile_size_train_val_test=(MS_SIZE['train'], MS_SIZE['val'], MS_SIZE['test']), 
                              cloud_sea_removal=CLOUD_SEA_REMOVAL, 
                              cloud_sea_weights_path=CLOUD_SEA_WEIGHTS_PATH, 
                              cloud_sea_pred_cutoff=CLOUD_SEA_PRED_CUTOFF,
                              cloud_sea_keep_rate=CLOUD_SEA_KEEP_RATE,
                              save_meta_to_disk=True)

In [5]:
if TILE_DENSITY_MAPS:
    for row in meta.iterrows():
        img_uid = row[0]
        density = tile_density_map(DATA_PATH_TILES, 
                                   row[1], 
                                   pan_or_ms='pan',
                                   density_dtype='uint8',
                                   write_to_disk=True,
                                   write_dir=DATA_PATH_TILES + '/density-maps', 
                                   write_filename=img_uid)
    # Plot last density
    plt.imshow(density)

In [6]:
if CALCULATE_STATS:
    train_tiles_mean, train_tiles_sd = mean_sd_of_train_tiles(DATA_PATH_TILES, 
                                                              sample_proportion=1.0, 
                                                              write_json=True)
else:
    train_tiles_mean, train_tiles_sd = read_mean_sd_json(DATA_PATH_TILES)

Loaded mean 341.3 and sd 128.4 from json file @ data/toulon-laspezia-tiles/e01/train_mean_sd.json


## 3. Data input pipeline from disk

### 3.1 Training set

In [7]:
SHUFFLE_BUFFER_SIZE = {'train': n_tiles['train'],  # 100
                       'val': n_tiles['val'],  # 100
                       'test': n_tiles['test']}  # 100

train_val_test = 'train'
sensor = SENSORS[train_val_test]
ds_train = {sensor: GeotiffDataset(tiles_path=DATA_PATH_TILES_P[train_val_test], 
                                   batch_size=BATCH_SIZE[train_val_test], 
                                   ms_tile_shape=(MS_SIZE[train_val_test], MS_SIZE[train_val_test], N_MS_BANDS), 
                                   pan_tile_shape=(PAN_SIZE[train_val_test], PAN_SIZE[train_val_test], N_PAN_BANDS),
                                   sensor=sensor,
                                   band_selection=MS_BANDS_WV02_IDXS, 
                                   mean_correction=train_tiles_mean,
                                   cache_memory=True,
                                   cache_file=str(DATA_PATH_TILES + '/ds_' + EXPERIMENT + '-'
                                                    + train_val_test + '-' + sensor + '_cache'), 
                                   repeat=True, 
                                   shuffle=True, 
                                   shuffle_buffer_size=SHUFFLE_BUFFER_SIZE[train_val_test])
           }
# Getting the scaled output range from the scaler. Needed to calculate PSNR and SSIM:
scaled_range = ds_train[sensor].get_scaler_output_range(print_ranges=True)

# Returning the actual tf.data.dataset object:
ds_train[sensor] = ds_train[sensor].get_dataset()
print(ds_train.keys())

Scaler ranges:
Input (uint) min, max: 0 2047
Input (uint) range: 2048
Output (float) range 1.2006480509994506
Output (float) min, max: -0.2000617970682984 1.0
dict_keys(['WV02'])


### 3.2 Validation set

In [8]:
# Validation set can have several sensors and is organized in a dictionary
# structure: ds_val = {sensor: dataset} ... ex: ds_val = {'WV02': dataset_with_only_WV02_images}
train_val_test = 'val'
ds_val = {}
for sensor in SENSORS[train_val_test]:
    if sensor == 'WV02':
        band_indices = MS_BANDS_WV02_IDXS
    elif sensor == 'GE01':
        band_indices = MS_BANDS_GE01_IDXS
    ds_val[sensor] = GeotiffDataset(tiles_path=DATA_PATH_TILES_P[train_val_test], 
                                    batch_size=BATCH_SIZE[train_val_test], 
                                    ms_tile_shape=(MS_SIZE[train_val_test], MS_SIZE[train_val_test], N_MS_BANDS), 
                                    pan_tile_shape=(PAN_SIZE[train_val_test], PAN_SIZE[train_val_test], N_PAN_BANDS),
                                    sensor=sensor,
                                    band_selection=band_indices, 
                                    mean_correction=train_tiles_mean,
                                    cache_memory=True,
                                    cache_file=str(DATA_PATH_TILES + '/ds_' + EXPERIMENT + '-'
                                                   + train_val_test + '-' + sensor + '_cache'), 
                                    repeat=True, 
                                    shuffle=True, 
                                    shuffle_buffer_size=SHUFFLE_BUFFER_SIZE[train_val_test])
    ds_val[sensor] = ds_val[sensor].get_dataset()
print(ds_val.keys())

dict_keys(['WV02', 'GE01'])


### 3.3 Test set

In [9]:
train_val_test = 'test'
ds_test = {}
for sensor in SENSORS[train_val_test]:
    if sensor == 'WV02':
        band_indices = MS_BANDS_WV02_IDXS
    elif sensor == 'GE01':
        band_indices = MS_BANDS_GE01_IDXS
    ds_test[sensor] = GeotiffDataset(tiles_path=DATA_PATH_TILES_P[train_val_test], 
                                     batch_size=BATCH_SIZE[train_val_test], 
                                     ms_tile_shape=(MS_SIZE[train_val_test], MS_SIZE[train_val_test], N_MS_BANDS), 
                                     pan_tile_shape=(PAN_SIZE[train_val_test], PAN_SIZE[train_val_test], N_PAN_BANDS),
                                     sensor=sensor,
                                     band_selection=band_indices, 
                                     mean_correction=train_tiles_mean,
                                     cache_memory=False,
                                     cache_file=str(DATA_PATH_TILES + '/ds_' + EXPERIMENT + '-'
                                                    + train_val_test + '-' + sensor + '_cache'), 
                                     repeat=False, 
                                     shuffle=False, 
                                     shuffle_buffer_size=SHUFFLE_BUFFER_SIZE[train_val_test])
    ds_test[sensor] = ds_test[sensor].get_dataset()
print(ds_test.keys())

dict_keys(['WV02', 'GE01'])


## 4. Build preliminary models

### 4.1 Bicubic baseline model

In [10]:
bicubic = build_deterministic_sr_model(upsample_factor=SR_FACTOR,
                                       resize_method='bicubic',
                                       loss='mean_absolute_error',
                                       metrics=('PSNR', 'SSIM'),
                                       scaled_range=scaled_range)

### 4.2 ESRGAN Generator model (pretrain version)

In [11]:
if PRETRAIN:
    pretrain_model =  build_generator(pretrain_or_gan='pretrain', 
                                      pretrain_learning_rate=PRETRAIN_LR, 
                                      pretrain_loss_l1_l2=PRETRAIN_LOSS,
                                      pretrain_beta_1=PRETRAIN_BETA_1, 
                                      pretrain_beta_2=PRETRAIN_BETA_2, 
                                      pretrain_metrics=('PSNR', 'SSIM'),
                                      scaled_range=scaled_range, 
                                      n_channels_in=N_MS_BANDS, 
                                      n_channels_out=N_PAN_BANDS, 
                                      height_width_in=None,  # None will make network image size agnostic
                                      n_filters=N_FILTERS, 
                                      n_blocks=N_BLOCKS)
    # pretrain_model.summary()

## 5. Pretraining with L1 loss

In [12]:
if PRETRAIN:
    history = pretrain_esrgan(generator=pretrain_model,
                              ds_train_dict=ds_train,
                              epochs=PRE_EPOCHS,
                              steps_per_epoch=PRE_TRAIN_STEPS,
                              initial_epoch=0,
                              validate=True,
                              ds_val_dict=ds_val,
                              val_steps=PRE_VAL_STEPS,
                              model_name=EXPERIMENT + '-pre',
                              tag=EXPERIMENT,
                              log_tensorboard=True,
                              tensorboard_logs_dir=LOGS_EXP_DIR + '/tb',
                              save_models=True,
                              models_save_dir=LOGS_EXP_DIR + '/models',
                              save_weights_only=True,
                              log_train_images=True,
                              n_train_image_batches=TRAIN_N_BATCHES_SAVE,
                              log_val_images=True,
                              n_val_image_batches=VAL_N_BATCHES_SAVE)

## 6. Build the full ESRGAN Model

In [13]:
if GAN_TRAIN:
    gan_model = build_esrgan_model(PRETRAIN_WEIGHTS_PATH,
                                   n_channels_in=N_MS_BANDS, 
                                   n_channels_out=N_PAN_BANDS, 
                                   n_filters=N_FILTERS, 
                                   n_blocks=N_BLOCKS, 
                                   pan_shape=(PAN_SIZE['train'], PAN_SIZE['train'], N_PAN_BANDS),
                                   G_lr=GAN_G_LR, 
                                   D_lr=GAN_D_LR, 
                                   G_beta_1=G_BETA_1, 
                                   G_beta_2=G_BETA_2, 
                                   D_beta_1=D_BETA_1, 
                                   D_beta_2=D_BETA_2,
                                   G_loss_pixel_w=G_LOSS_PIXEL_W, 
                                   G_loss_pixel_l1_l2='l1',
                                   G_loss_percep_w=G_LOSS_PERCEP_W, 
                                   G_loss_percep_l1_l2='l1', 
                                   G_loss_percep_layer=54,
                                   G_loss_percep_before_act=True,
                                   G_loss_generator_w=G_LOSS_GENERATOR_W,
                                   metric_reg=False, 
                                   metric_ma=METRIC_MA, 
                                   metric_niqe=METRIC_NIQE, 
                                   ma_niqe_proportion=MA_NIQE_PROPORTION,
                                   matlab_wd_path='modules/matlab',
                                   scale_mean=train_tiles_mean, 
                                   scaled_range=scaled_range, 
                                   shave_width=SHAVE_WIDTH)

## 7. GAN training

In [ ]:
if GAN_TRAIN:
    history = gan_train_esrgan(esrgan_model=gan_model,
                               ds_train_dict=ds_train,
                               epochs=GAN_EPOCHS,
                               steps_per_epoch=GAN_TRAIN_STEPS,
                               initial_epoch=0,
                               validate=True,
                               ds_val_dict=ds_val,
                               val_steps=GAN_VAL_STEPS,
                               model_name=EXPERIMENT + '-gan',
                               tag=EXPERIMENT,
                               log_tensorboard=True,
                               tensorboard_logs_dir=LOGS_EXP_DIR + '/tb',
                               save_models=True,
                               models_save_dir=LOGS_EXP_DIR + '/models',
                               save_weights_only=True,
                               log_train_images=True,
                               n_train_image_batches=TRAIN_N_BATCHES_SAVE,
                               log_val_images=True,
                               n_val_image_batches=VAL_N_BATCHES_SAVE)

32 images from train-WV02 will be logged at each epoch
16 images from val-WV02 will be logged at each epoch
16 images from val-GE01 will be logged at each epoch
Callbacks: [<modules.logging.SimpleEsrganCheckpoint object at 0x000001AC9D348108>, <tensorflow.python.keras.callbacks.TensorBoard object at 0x000001AC9D33E6C8>, <modules.logging.MultipleValSetsCallback object at 0x000001AD19DFC4C8>, <modules.logging.LrHrSrImageCallback object at 0x000001ACA3FB55C8>, <modules.logging.LrHrSrImageCallback object at 0x000001AD1761FB08>]
Epoch 1/400
1000/1000 [==============================] - 872s 872ms/step - G_pixel_loss: 1.0645e-04 - G_perceptual_loss: 1.9385 - G_generator_loss: 0.0227 - G_loss_total: 1.9614 - D_loss_total: 0.1929 - PSNR: 39.0872 - SSIM: 0.9128
Epoch 2/400
1000/1000 [==============================] - 887s 887ms/step - G_pixel_loss: 1.0964e-04 - G_perceptual_loss: 1.8598 - G_generator_loss: 0.0385 - G_loss_total: 1.8983 - D_loss_total: 0.0333 - PSNR: 38.6199 - SSIM: 0.9128
Epoch 

1000/1000 [==============================] - 836s 836ms/step - G_pixel_loss: 1.1053e-04 - G_perceptual_loss: 1.7436 - G_generator_loss: 0.0458 - G_loss_total: 1.7895 - D_loss_total: 0.0519 - PSNR: 38.6589 - SSIM: 0.9039
Epoch 34/400
1000/1000 [==============================] - 835s 835ms/step - G_pixel_loss: 1.0946e-04 - G_perceptual_loss: 1.7303 - G_generator_loss: 0.0434 - G_loss_total: 1.7738 - D_loss_total: 0.0563 - PSNR: 38.7780 - SSIM: 0.9046
Epoch 35/400
1000/1000 [==============================] - 836s 836ms/step - G_pixel_loss: 1.0948e-04 - G_perceptual_loss: 1.7230 - G_generator_loss: 0.0439 - G_loss_total: 1.7670 - D_loss_total: 0.0555 - PSNR: 38.7008 - SSIM: 0.9048
Epoch 36/400
1000/1000 [==============================] - 846s 846ms/step - G_pixel_loss: 1.1016e-04 - G_perceptual_loss: 1.7271 - G_generator_loss: 0.0434 - G_loss_total: 1.7706 - D_loss_total: 0.0465 - PSNR: 38.6688 - SSIM: 0.9041
Epoch 37/400
1000/1000 [==============================] - 835s 835ms/step - G_pix

1000/1000 [==============================] - 836s 836ms/step - G_pixel_loss: 1.0906e-04 - G_perceptual_loss: 1.6600 - G_generator_loss: 0.0512 - G_loss_total: 1.7113 - D_loss_total: 0.0317 - PSNR: 38.7480 - SSIM: 0.9048
Epoch 69/400
1000/1000 [==============================] - 836s 836ms/step - G_pixel_loss: 1.1010e-04 - G_perceptual_loss: 1.6664 - G_generator_loss: 0.0476 - G_loss_total: 1.7142 - D_loss_total: 0.0562 - PSNR: 38.7118 - SSIM: 0.9035
Epoch 70/400
1000/1000 [==============================] - 836s 836ms/step - G_pixel_loss: 1.0836e-04 - G_perceptual_loss: 1.6610 - G_generator_loss: 0.0518 - G_loss_total: 1.7129 - D_loss_total: 0.0375 - PSNR: 38.8497 - SSIM: 0.9055
Epoch 71/400
1000/1000 [==============================] - 835s 835ms/step - G_pixel_loss: 1.1001e-04 - G_perceptual_loss: 1.6699 - G_generator_loss: 0.0473 - G_loss_total: 1.7174 - D_loss_total: 0.0550 - PSNR: 38.7012 - SSIM: 0.9035
Epoch 72/400
1000/1000 [==============================] - 835s 835ms/step - G_pix

1000/1000 [==============================] - 837s 837ms/step - G_pixel_loss: 1.1052e-04 - G_perceptual_loss: 1.6104 - G_generator_loss: 0.0526 - G_loss_total: 1.6631 - D_loss_total: 0.0261 - PSNR: 38.6809 - SSIM: 0.9031
Epoch 104/400
1000/1000 [==============================] - 838s 838ms/step - G_pixel_loss: 1.1006e-04 - G_perceptual_loss: 1.6182 - G_generator_loss: 0.0538 - G_loss_total: 1.6721 - D_loss_total: 0.0517 - PSNR: 38.7045 - SSIM: 0.9034
Epoch 105/400
1000/1000 [==============================] - 837s 837ms/step - G_pixel_loss: 1.0970e-04 - G_perceptual_loss: 1.6089 - G_generator_loss: 0.0520 - G_loss_total: 1.6611 - D_loss_total: 0.0282 - PSNR: 38.7024 - SSIM: 0.9032
Epoch 106/400
1000/1000 [==============================] - 837s 837ms/step - G_pixel_loss: 1.0948e-04 - G_perceptual_loss: 1.5901 - G_generator_loss: 0.0550 - G_loss_total: 1.6452 - D_loss_total: 0.0363 - PSNR: 38.7355 - SSIM: 0.9040
Epoch 107/400
1000/1000 [==============================] - 837s 837ms/step - G

1000/1000 [==============================] - 836s 836ms/step - G_pixel_loss: 1.0965e-04 - G_perceptual_loss: 1.5498 - G_generator_loss: 0.0539 - G_loss_total: 1.6038 - D_loss_total: 0.0327 - PSNR: 38.7383 - SSIM: 0.9036
Epoch 139/400
1000/1000 [==============================] - 835s 835ms/step - G_pixel_loss: 1.1093e-04 - G_perceptual_loss: 1.5629 - G_generator_loss: 0.0553 - G_loss_total: 1.6183 - D_loss_total: 0.0308 - PSNR: 38.6052 - SSIM: 0.9022
Epoch 140/400
1000/1000 [==============================] - 836s 836ms/step - G_pixel_loss: 1.0866e-04 - G_perceptual_loss: 1.5448 - G_generator_loss: 0.0503 - G_loss_total: 1.5952 - D_loss_total: 0.0381 - PSNR: 38.8549 - SSIM: 0.9045
Epoch 141/400
1000/1000 [==============================] - 836s 836ms/step - G_pixel_loss: 1.1162e-04 - G_perceptual_loss: 1.5702 - G_generator_loss: 0.0563 - G_loss_total: 1.6266 - D_loss_total: 0.0290 - PSNR: 38.5236 - SSIM: 0.9014
Epoch 142/400
1000/1000 [==============================] - 835s 835ms/step - G

1000/1000 [==============================] - 836s 836ms/step - G_pixel_loss: 1.0849e-04 - G_perceptual_loss: 1.5020 - G_generator_loss: 0.0508 - G_loss_total: 1.5529 - D_loss_total: 0.0438 - PSNR: 38.8574 - SSIM: 0.9045
Epoch 174/400
1000/1000 [==============================] - 836s 836ms/step - G_pixel_loss: 1.1116e-04 - G_perceptual_loss: 1.5147 - G_generator_loss: 0.0557 - G_loss_total: 1.5705 - D_loss_total: 0.0294 - PSNR: 38.6334 - SSIM: 0.9015
Epoch 175/400
1000/1000 [==============================] - 836s 836ms/step - G_pixel_loss: 1.1013e-04 - G_perceptual_loss: 1.5207 - G_generator_loss: 0.0619 - G_loss_total: 1.5827 - D_loss_total: 0.0305 - PSNR: 38.6607 - SSIM: 0.9031
Epoch 176/400
1000/1000 [==============================] - 836s 836ms/step - G_pixel_loss: 1.1001e-04 - G_perceptual_loss: 1.5230 - G_generator_loss: 0.0525 - G_loss_total: 1.5756 - D_loss_total: 0.0262 - PSNR: 38.6672 - SSIM: 0.9025
Epoch 177/400
1000/1000 [==============================] - 836s 836ms/step - G

1000/1000 [==============================] - 835s 835ms/step - G_pixel_loss: 1.1027e-04 - G_perceptual_loss: 1.4966 - G_generator_loss: 0.0583 - G_loss_total: 1.5550 - D_loss_total: 0.0252 - PSNR: 38.7003 - SSIM: 0.9020
Epoch 209/400
1000/1000 [==============================] - 836s 836ms/step - G_pixel_loss: 1.0941e-04 - G_perceptual_loss: 1.4883 - G_generator_loss: 0.0527 - G_loss_total: 1.5410 - D_loss_total: 0.0294 - PSNR: 38.7774 - SSIM: 0.9027
Epoch 210/400
1000/1000 [==============================] - 836s 836ms/step - G_pixel_loss: 1.0949e-04 - G_perceptual_loss: 1.4853 - G_generator_loss: 0.0632 - G_loss_total: 1.5487 - D_loss_total: 0.0165 - PSNR: 38.7647 - SSIM: 0.9027
Epoch 211/400
1000/1000 [==============================] - 836s 836ms/step - G_pixel_loss: 1.0941e-04 - G_perceptual_loss: 1.4619 - G_generator_loss: 0.0682 - G_loss_total: 1.5301 - D_loss_total: 0.0286 - PSNR: 38.7880 - SSIM: 0.9032
Epoch 212/400
1000/1000 [==============================] - 836s 836ms/step - G

1000/1000 [==============================] - 838s 838ms/step - G_pixel_loss: 1.1024e-04 - G_perceptual_loss: 1.4498 - G_generator_loss: 0.0729 - G_loss_total: 1.5228 - D_loss_total: 0.0110 - PSNR: 38.6940 - SSIM: 0.9020
Epoch 244/400
1000/1000 [==============================] - 837s 837ms/step - G_pixel_loss: 1.1067e-04 - G_perceptual_loss: 1.4528 - G_generator_loss: 0.0636 - G_loss_total: 1.5165 - D_loss_total: 0.0401 - PSNR: 38.6292 - SSIM: 0.9016
Epoch 245/400
1000/1000 [==============================] - 837s 837ms/step - G_pixel_loss: 1.1016e-04 - G_perceptual_loss: 1.4470 - G_generator_loss: 0.0648 - G_loss_total: 1.5119 - D_loss_total: 0.0132 - PSNR: 38.6946 - SSIM: 0.9021
Epoch 246/400
1000/1000 [==============================] - 837s 837ms/step - G_pixel_loss: 1.1002e-04 - G_perceptual_loss: 1.4540 - G_generator_loss: 0.0710 - G_loss_total: 1.5251 - D_loss_total: 0.0156 - PSNR: 38.6973 - SSIM: 0.9031
Epoch 247/400
1000/1000 [==============================] - 837s 837ms/step - G

1000/1000 [==============================] - 836s 836ms/step - G_pixel_loss: 1.1018e-04 - G_perceptual_loss: 1.4213 - G_generator_loss: 0.0698 - G_loss_total: 1.4911 - D_loss_total: 0.0054 - PSNR: 38.6907 - SSIM: 0.9027
Epoch 279/400
1000/1000 [==============================] - 836s 836ms/step - G_pixel_loss: 1.1125e-04 - G_perceptual_loss: 1.4293 - G_generator_loss: 0.0707 - G_loss_total: 1.5002 - D_loss_total: 0.0210 - PSNR: 38.6276 - SSIM: 0.9012
Epoch 280/400
1000/1000 [==============================] - 836s 836ms/step - G_pixel_loss: 1.1101e-04 - G_perceptual_loss: 1.4275 - G_generator_loss: 0.0664 - G_loss_total: 1.4941 - D_loss_total: 0.0221 - PSNR: 38.6795 - SSIM: 0.9006
Epoch 281/400
1000/1000 [==============================] - 836s 836ms/step - G_pixel_loss: 1.0927e-04 - G_perceptual_loss: 1.4236 - G_generator_loss: 0.0672 - G_loss_total: 1.4908 - D_loss_total: 0.0261 - PSNR: 38.7811 - SSIM: 0.9033
Epoch 282/400
1000/1000 [==============================] - 837s 837ms/step - G

## 8. Inspection of results